In [1]:
import pandas as pd

import sys
sys.path.append('../pycore/')
from utils import metabolites_dict_from_reaction_equation_RBA

In [2]:
#### LOAD INPUTS AND PARAMETERS
# Stoichiometry
df_eqn = pd.read_excel('../scRBA_build_GAMS_model/model/RBA_stoichiometry.xlsx')
df_eqn.index = df_eqn.id.to_list()

#### Assemble list of proteins and enzymes

In [3]:
met_list_raw = []
for i in df_eqn.index:
    met_dict = metabolites_dict_from_reaction_equation_RBA(df_eqn.reaction[i])
    met_list_raw += met_dict.keys()

met_list = sorted(list(set(met_list_raw)))
met_list = [i for i in met_list if i != '']
met_list = [i for i in met_list if i.split('-', maxsplit=1)[0] in ['PRO', 'ENZ']]

met_list_write = ["'" + i + "'" for i in met_list]
met_list_write = ['/'] + met_list_write + ['/']

In [4]:
with open('./enz_from_proteome/pro_and_enz.txt', 'w') as f:
    f.write('\n'.join(met_list_write))

#### Assemble list of reactions regarding proteins-to-enzymes balance

All reactions

In [5]:
rxn_list = df_eqn.id.to_list()
rxn_list = [i for i in rxn_list if i.split('-')[0] in ['ENZSYN', 'ENZLOAD']]

for met in met_list:
    stype,sid = met.split('-', maxsplit=1)
    if stype == 'PRO':
        rxn_list.append('PROIN-' + sid)
        
rxn_list_write = ["'" + i + "'" for i in rxn_list]
rxn_list_write = ['/'] + rxn_list_write + ['/']

In [7]:
with open('./enz_from_proteome/rxns_pro_and_enz.txt', 'w') as f:
    f.write('\n'.join(rxn_list_write))

Enzyme synthesis and enzyme load reactions

In [8]:
rxns_enz = df_eqn.id.to_list()
rxns_enz = [i for i in rxns_enz if i.split('-')[0] in ['ENZSYN', 'ENZLOAD']]
        
rxns_enz = ["'" + i + "'" for i in rxns_enz]
rxns_enz = ['/'] + rxns_enz + ['/']

In [9]:
with open('./enz_from_proteome/rxns_enz.txt', 'w') as f:
    f.write('\n'.join(rxns_enz))

#### Assemble stoichiometry

In [12]:
rxns_enz = df_eqn.id.to_list()
rxns_enz = [i for i in rxns_enz if i.split('-')[0] in ['ENZSYN', 'ENZLOAD']]

sij = []
for i in rxns_enz:
    x = metabolites_dict_from_reaction_equation_RBA(df_eqn.reaction[i])
    met_dict = dict()
    for k,v in x.items():
        if k == '':
            continue
        if v.is_integer():
            met_dict[k] = int(v)
        else:
            met_dict[k] = v
            
    for k,v in met_dict.items():
        sij.append("'" + k + "'.'" + df_eqn.id[i] + "' " + str(v))
    
for met in met_list:
    stype,sid = met.split('-', maxsplit=1)
    if stype == 'PRO':
        sij.append("'" + met + "'.'" + "PROIN-" + sid + "' 1")

sij = ['/'] + sij + ['/']

In [14]:
with open('./enz_from_proteome/sij_pro_and_enz.txt', 'w') as f:
    f.write('\n'.join(sij))